In [1]:
import numpy as np
import tensorflow as tf
import gpflow

from vgpmp.prior import MaternPositionPrior
from vgpmp.likelihood import PlanningLikelihood
from vgpmp.models import VGPPlanner

DTYPE = tf.float64
gpflow.config.set_default_float(DTYPE)
gpflow.config.set_default_jitter(1e-5)

2025-12-01 07:17:33.610871: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-01 07:17:33.635442: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-01 07:17:33.635472: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-01 07:17:33.636196: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-01 07:17:33.640561: I tensorflow/core/platform/cpu_feature_guar

In [2]:
dof = 2
t0  = 0.0                   # start time
tN  = 1.0                   # end time
num_interior = 64           # number of interior support points
q_start = np.array([0.0, 0.0], dtype=np.float64)
q_goal  = np.array([10.0, 10.0], dtype=np.float64)

# Times excluding start and end
t_interior = np.linspace(t0, tN, num_interior + 2)[1:-1]    # shape (num_interior,)
X_in = t_interior.reshape(-1, 1).astype(np.float64)         # shape (K,1)

# Dummy Y_in if your likelihood ignores Y
Y_in = np.zeros((num_interior, dof), dtype=np.float64)      # shape (K, dof)

X_ep = np.array([[t0], [tN]], dtype=np.float64)             # shape (2,1)
Y_ep = np.stack([q_start, q_goal], axis=0)                  # shape (2,dof)

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

def plot_mean_and_obstacle(model, X):
    # Model mean
    f_mean, f_var = model.predict_f(X)
    mean_pos = f_mean.numpy()

    # Obstacle params
    cx, cy = model.likelihood.center.numpy()
    r = float(model.likelihood.radius.numpy())
    eps = float(model.likelihood.epsilon.numpy())
    L = float(model.likelihood.grid_size.numpy())

    fig, ax = plt.subplots()
    ax.plot(mean_pos[:, 0], mean_pos[:, 1], "-o", label="mean trajectory", color="C0")

    # Obstacle circle and (optional) safety buffer R+epsilon
    circle = plt.Circle((cx, cy), r, color="red", alpha=0.25, label="obstacle")
    buffer = plt.Circle((cx, cy), r + eps, color="red", alpha=0.1, fill=False, linestyle="--", label="radius + epsilon")
    ax.add_patch(circle)
    ax.add_patch(buffer)

    ax.set_xlim(0, L)
    ax.set_ylim(0, L)
    ax.set_aspect("equal", adjustable="box")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.legend()
    ax.set_title("Mean trajectory and circular obstacle")
    plt.show()

In [4]:
prior = MaternPositionPrior(dof, M=num_interior, t_min=t0, t_max=tN, matern_family="52")
planning_lik = PlanningLikelihood(latent_dim=dof, obs_scale=2.0, temperature=1.0)
model = VGPPlanner(prior, planning_lik)

2025-12-01 07:17:35.909112: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-01 07:17:35.932673: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-01 07:17:35.932835: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:
data = {
    "interior": (X_in, Y_in),
    "endpoints": (X_ep, Y_ep),
}
loss_fn = lambda: -model.elbo(data)

variational_vars = [(model.q_mu, model.q_sqrt)]
natgrad_opt = gpflow.optimizers.NaturalGradient(gamma=0.1)

adam_vars = model.trainable_variables
adam_opt = tf.optimizers.Adam(0.01)

@tf.function
def optimization_step(data):
    natgrad_opt.minimize(loss_fn(data), variational_vars)
    adam_opt.minimize(loss_fn(data), adam_vars)

In [ ]:
num_iters = 100
for _ in range(num_iters):
    optimization_step(data)

2025-12-01 07:22:53.029952: W tensorflow/core/kernels/linalg/cholesky_op_gpu.cu.cc:205] Cholesky decomposition was not successful for batch 0. The input might not be valid. Filling lower-triangular output with NaNs.
2025-12-01 07:22:53.030016: W tensorflow/core/kernels/linalg/cholesky_op_gpu.cu.cc:205] Cholesky decomposition was not successful for batch 1. The input might not be valid. Filling lower-triangular output with NaNs.
2025-12-01 07:22:53.030649: E tensorflow/core/kernels/check_numerics_op.cc:293] abnormal_detected_host @0x7e178b20a900 = {1, 0} gpflow.Parameter: the value to be assigned is incompatible with this parameter's transform (the corresponding unconstrained value has NaN or Inf) and hence cannot be assigned.


InvalidArgumentError: Graph execution error:

Detected at node NaturalGradient/natural_gradient_steps/VerifyFinite/CheckNumerics defined at (most recent call last):
  File "/home/abbas/miniconda3/envs/gpmp/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/home/abbas/miniconda3/envs/gpmp/lib/python3.9/runpy.py", line 87, in _run_code

  File "/home/abbas/miniconda3/envs/gpmp/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/abbas/miniconda3/envs/gpmp/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/abbas/miniconda3/envs/gpmp/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/abbas/miniconda3/envs/gpmp/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/home/abbas/miniconda3/envs/gpmp/lib/python3.9/asyncio/base_events.py", line 601, in run_forever

  File "/home/abbas/miniconda3/envs/gpmp/lib/python3.9/asy

In [ ]:
X_dense = np.linspace(t0, tN, 100, dtype=np.float64).reshape(-1,1)
plot_mean_and_obstacle(model, X_dense)